# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  # f'(a) = -3*a**2 - 0.5*a**(-0.5)
  #       = -3*a**2 - (1/(4 * sqrt(a)))
  
  # f'(b) = 3 * cos(3*b) + 2.5 * b ** 1.5
  
  # f'(c) = 1 / c**2
  
  dfda = -3*a**2 - 0.5*a**(-0.5)
  dfdb = 3*cos(3*b) + 2.5*(b**1.5)
  dfdc = 1 / (c**2)
  return [dfda, dfdb, dfdc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [7]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

h = 0.00001
num_dfda = (f(2+h, 3, 4) - f(2, 3, 4)) / h
num_dfdb = (f(2, 3+h, 4) - f(2, 3, 4)) / h
num_dfdc = (f(2, 3, 4+h) - f(2, 3, 4)) / h

numerical_grad = [num_dfda, num_dfdb, num_dfdc]


for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


WRONG! for dim 0: expected -12.353553390593273, yours returns -12.353612948956536
WRONG! for dim 1: expected 10.25699027111255, yours returns 10.257004202163245
OK for dim 2: expected 0.0625, yours returns 0.06249984378925432


In [28]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h = 0.00001
num_dfda_2 = (f(2+h, 3, 4) - f(2-h, 3, 4)) / (2*h)
num_dfdb_2 = (f(2, 3+h, 4) - f(2, 3-h, 4)) / (2*h)
num_dfdc_2 = (f(2, 3, 4+h) - f(2, 3, 4-h)) / (2*h)

numerical_grad2 = [num_dfda_2, num_dfdb_2, num_dfdc_2]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


## section 2: support for softmax

In [ ]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data}, op={self._op}, label={self.label})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out
  
  def __radd__(self, other):
    return self + other
  
  def __sub__(self, other):
    return self + (-other)
  
  def __rsub__(self, other):
    return self + (-other)
  
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), "*")
    
    def _backward():
      self.grad += out.grad * other.data
      other.grad += out.grad * self.data
    out._backward = _backward
    
    return out
  
  def __rmul__(self, other):
    return self * other
  
  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data ** other, (self, Value(other)), "**")
    
    def _backward():
      self.grad += (other * self.data ** (other-1)) * out.grad 
    out._backward = _backward
    
    return out
  
  def __truediv__(self, other):
    return self * other ** -1
  
  def __rtruediv__(self, other):
    return other * self ** -1
  
  def exp(self):
    out = Value(exp(self.data), (self, ), "exp")
    
    def _backward():
      self.grad += out.grad * out.data
    out._backward = _backward
    
    return out
  
  def __neg__(self):
    return -1 * self
  
  def log(self):
    out = Value(log(self.data), (self, ), "log")
    
    def _backward():
      self.grad += (1 / self.data) * out.grad
    out._backward = _backward
    
    return out

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [105]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges
    
def draw_dot(root):
    dot = Digraph(format="svg", graph_attr={"rankdir": "LR"})
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape = "record")
        if n._op:
            dot.node(name = uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [139]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits) # normalize
loss = -probs[3].log() # take log and take the label matching index (dim 3 acts as the label for this input example)
loss.backward()
print(loss.data)


ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


## NLL, CE, BCE etc..


#### Negative Log Likelihood (in a binary classification setting)

$$
\begin{equation*}
\begin{align}
\mathbb{P}(\mathcal{D} \mid \theta) = \prod_{i=1}^{n} \hat{y}_{\theta, i}^{y_i} (1 - \hat{y}_{\theta, i})^{1 - y_i}  \\
\log \mathbb{P}(\mathcal{D} \mid \theta) = \sum_{i=1}^{n} \left( y_i \log \hat{y}_{\theta, i} + (1 - y_i) \log (1 - \hat{y}_{\theta, i}) \right)  \\
l(\theta) = -\sum_{i=1}^{n} \left( y_i \log \hat{y}_{\theta, i} + (1 - y_i) \log (1 - \hat{y}_{\theta, i}) \right)  
\end{align}
\end{equation*}
$$

* Start with predicted probabilities for the positive class (y_hat). If we were given raw prediction values, apply sigmoid to make it a probability.
* Compute the probabilities for the negative class (1-y_hat).
* Compute the log probabilities.
* Summing up the log probabilities associated with the true labels.

In summary, **normalize**, **take log**, **sum up the log probs of the true labels**.  

Sine the logarithmic function is monotonic, maximizing the likelihood is the same as maximizing the log likelihood. However, since minimizing loss makes more sense, we take the negative of the log likelihood function.


#### Negative Log Likelihood (in a multiclass setting)

* softmax is used to normalize the probabilities


In [146]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

logits = torch.tensor([0.0, 3.0, -2.0, 1.0]).double() ; logits.requires_grad = True
m = torch.nn.Softmax(dim=0)
probs = m(logits)
loss = -torch.log(probs[3])
loss.backward()

logits.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)